In [ ]:
## Install required packages
# !pip install torchsummary

In [ ]:
# Import required packages
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torchvision import datasets, transforms
from tqdm.auto import tqdm

In [ ]:
# Class to define the NN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # # Reference Links
        # https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html
        # https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
        # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
        # https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
        # https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
        # https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        # https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html
        # https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html
        # https://pytorch.org/docs/stable/generated/torch.Tensor.view.html

        # Using Sequential API to define the model as it seems to be more readable

        # General Notes

        # ReLU used after every Convolution layer
        # Batch Normalization used after every Convolution layer
        # Dropout used after every block/ layer
        # Max Pooling preferably used after every block
        # GAP used at the end of the model

        # Sequence in a block could be as follows
        # Convolution Layer -> ReLU -> Batch Normalization -> Max Pooling -> Dropout

        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.1),
        )

        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.15),
        )

        # Block 3
        self.block3 = nn.Sequential(
            nn.Conv2d(16, 32, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
        )

        # Gap layer
        self.block4 = nn.Sequential(
            # nn.AdaptiveAvgPool2d(1),
            nn.AvgPool2d(kernel_size=3),
            nn.Flatten(),
            nn.Linear(32, 10),
        )

    def forward(self, x):
        x = self.block1(x)
        # print(x.shape)

        x = self.block2(x)
        # print(x.shape)

        x = self.block3(x)
        # print(x.shape)

        x = self.block4(x)
        # print(x.shape)

        return F.log_softmax(x)

In [ ]:
# Check if GPU is available and set device accordingly
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Move model to device and print summary given image size
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

In [ ]:
# Set seed for consistent results
torch.manual_seed(1)
# Set batch size
batch_size = 128

# Set kwargs for GPU if available
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

# Load train and test data and apply required transformations
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../data",
        train=False,
        transform=transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

In [ ]:
# Train the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc=f"loss={loss.item()} batch_id={batch_idx}")


# Test the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    # print test accuracy and loss
    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )

In [23]:
# Send model to device
model = Net().to(device)
# Set optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Specify the number of epochs
num_epochs = 20

# Train and test the model for specified number of epochs
for epoch in range(1, num_epochs + 1):
    if epoch == 1:
        print(f"\n\nBatch size: {batch_size}, Total epochs: {num_epochs}\n\n")

    print(f"\nEpoch {epoch}:\n")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

torch.Size([128, 16, 12, 12])
torch.Size([128, 16, 5, 5])
torch.Size([128, 32, 3, 3])
torch.Size([128, 10])
torch.Size([128, 16, 12, 12])
torch.Size([128, 16, 5, 5])
torch.Size([128, 32, 3, 3])
torch.Size([128, 10])
torch.Size([128, 16, 12, 12])
torch.Size([128, 16, 5, 5])
torch.Size([128, 32, 3, 3])
torch.Size([128, 10])
torch.Size([128, 16, 12, 12])
torch.Size([128, 16, 5, 5])
torch.Size([128, 32, 3, 3])
torch.Size([128, 10])
torch.Size([16, 16, 12, 12])
torch.Size([16, 16, 5, 5])
torch.Size([16, 32, 3, 3])
torch.Size([16, 10])

Test set: Average loss: 0.0841, Accuracy: 9784/10000 (98%)


Epoch 2:



  0%|          | 0/469 [00:01<?, ?it/s]


KeyboardInterrupt: 